<p style="page-break-after:always;"></p>

# Forrestania (Joint Grav/Mag)

This uses public data downloaded from the [Geological Survey of Western Australia](https://portal.ga.gov.au/persona/gadds) over the Forrestania geological province.


## Geological setting



## Data

We first need to consolidate the data into a usable format. We use [Geoscience ANALYST](https://www.mirageoscience.com/mining-industry-software/geoscience-analyst/) to import and convert the original file formats into `geoh5`. This can be done with a simple `drag & drop` to the viewport. The data bundle includes

 - Airborne magnetic survey: `60472_AOI4.gdb`
 - Ground gravity survey: `Forrestania_Gravity_Station_trim_.csv`
 - Digital Elevation Model (DEM): `Forrestania_SRTM1 Australia_MGA50.ers`

Once imported, the data can be activated for visualization.  

```{figure} ./images/data_import.png
---
name: data_import

---
Data imported from their original format to `geoh5` for visualization.
```

### Processing

Note that both the DEM and airborne magnetic survey were imported at an arbitrary 0 m elevation and do not reflect the true position.
We first need to modify our object such that the data is accurately located in 3D space. 

#### Step 1: Convert the 2D DEM into a 3D surface
```{figure} ./images/create_surface.png
```

#### Step 2a: Transfer the DEM data onto the airborne survey (curve). 
```{figure} ./images/transfer_dem.png
```

#### Step 2b: Re-assign the Z channel to the DEM + radar height.
```{figure} ./images/drape_mag.png
```



## Gravity

We begin our analysis with the ground gravity survey. We will attempt to estimate the shape and depth of the causative body at depth.

### Inputs

Gravity data are sensitive to variations in the density of rocks. The data were already processed and provided as terrain-corrected (2.67 g/cc) residual data. We can therefore go directly to the inversion phase. 

```{figure} ./images/gravity_data.png
```

### Inversion options

The following options are set for the inversion.

```{figure} ./images/gravity_setup.png
```
- Assign the data uncertainty to a quarter of the standard deviation (0.05 mGal). This is a somewhat conservative estimate that takes into account both the instrumental and numerical noise.
- Use a `0 g/cc` reference value for the model. This assumes that the signal from the background density was accurately subtracted from the data.
- Leave the `Mesh` option unchecked. We will let the code create an octree mesh based on the average data separation and extent.

### Results

After completion of the inversion, we load the results and assess convergence and data fit.

```{figure} ./images/grav_predicted.png
```

We note the following:

 - It took 13 iterations to converge to the target misfit.
 - The following 12 iterations have increased the model complexity ($phi_m$) but preserved the data fit ($phi_d$).
 - Most of the important gravity anomalies appear to be reproduced by the final model.

#### Smooth solution (L2-norm)

```{figure} ./images/grav_l2_model.png
```

The smooth solution (iteration 13) indicates the presence of a dense anomaly extending at depth with density contrasts ranging from [-0.02, 0.05] g/cc. The negative density contrasts appear to be localized around the main positive anomaly, likely due to the smoothness constraint and the lack of `bound` constraints. Some smaller "fuzzy" anomalies are visible at the margins.  

#### Compact solution (L0-norm)

```{figure} ./images/grav_lp_model.png
```

As an alternative solution, the compact model (iteration 25) recovers a well-defined dense body within a mostly uniform background. Density contrasts have substantially increased in the range of [0.0, 0.56] g/cc.

<p style="page-break-after:always;"></p>

## Magnetics

We follow up with the processing of the airborne magnetic data. The survey was conducted between February and March 1988.

### Inputs

The source of the magnetic signal can generally be attributed to the presence or destruction of magnetic minerals in rocks (mainly magnetite). Magnetometers measure the Total Magnetic Intensity (TMI), which includes both the primary (source) field and secondary fields (signal) from the local geology. The first step is to remove the primary (inducing) field at the time and location of acquisition.

#### Step 1: Set the coordinate system of the survey
```{figure} ./images/mag_coordinates.png
```

#### Step 2: Lookup/remove IGRF 

The inversion routine requires Residual Magnetic Intensity (RMI), so we need to subtract the ambient field (IGRF). 
```{figure} ./images/mag_igrf.png
```

The **inducing field parameters** at the time and location of the survey are

- Magnitude: 59127 nT
- Inclination: -66.9 $\textdegree$
- Declination: -1.05 $\textdegree$  

#### Step 3: Detrend
The local background field appears to be slightly lower (~112 nT) than the computed IGRF model, as most of the data away from the main anomaly are below 0. To avoid modelling this background trend, we can remove the median value. 

```{figure} ./images/mag_detrend.png
```

#### Step 4: Downsampling

We can downsample the data along line to reduce the computation cost and bring the resolution closer to the gravity survey. Since the average flight height of this survey was 60 m, we can confidently downsample the lines by the same amount without affecting the wavelengths contained in the data.

```{figure} ./images/mag_downsample.png
```

### Create a mesh

This time, we will create a mesh using the [Octree Mesh Creation](mesh-design) application. In preparation for the joint inversion, we will simply modify the gravity mesh parameters.

#### Step 1: Load the octree parameters
Inspect the files attached to the gravity mesh, and load the `ui.json`.

#### Step 2: Modify the refinements
Change the input object for `Refinement A` for the curve object of the magnetic survey.

#### Step 3: Run and load the results

```{figure} ./images/mag_mesh.png
```

By doing so, we guarantee that the extent and base cell size remain the same - only the refinements will be catered to the magnetic survey.
 
### Inversion options

The following options are set for the inversion.

```{figure} ./images/mag_setup.png
```

- Set the inducing field parameters as described above.
- Assign the data uncertainty to a quarter of the standard deviation (125 nT).
- Use a `0 SI` reference value for the model. This assumes that we removed all background signal from the data.
- Assign the newly created octree mesh object.
- Leave the default lower bound value to 0 SI (non-negative). 

### Results

After completion of the inversion, we load the results and assess convergence and data fit.

```{figure} ./images/mag_predicted.png
```

We note the following:

 - It took 5 iterations to converge to the target misfit.
 - The following 21 iterations have increased the model complexity ($phi_m$) but preserved the data fit ($phi_d$).
 - Small anomalies are under-fitted, which could warrant a second inversion with lower uncertainties.

#### Smooth solution (L2-norm)

```{figure} ./images/mag_l2_model.png
```

The smooth solution (iteration 5) indicates the presence of a susceptible body at depth. The shape and position  of the density model - including the presence of smaller "fuzzy" anomalies at the margins.  

#### Compact solution (L0-norm)

```{figure} ./images/mag_lp_model.png
```

As an alternative solution, the compact model (iteration 25) recovers a well-defined dense body within a mostly uniform background. Density contrasts have substantially increased in the range of [0.0, 0.56] g/cc.
